In [2]:
spark

org.apache.spark.sql.SparkSession@387ebf61

# 이벤트 시간과 상태 기반 처리 
+ 이벤트 시간 처리 
    + 이벤트 시간 : 데이터에 기록된 시간 기준 처리 
    + 처리 시간 : 스트림 처리 시스템이 데이터를 실제로 수신한 시간 
+ 상태 기반 처리 
    + 중간 처리 정보를 사용하거나 갱신하는 경우에 필요(마이크로 배치 혹은 레코드 단위 처리) 
    + 스파크는 상태 저장소를 제공 
+ 임의적인 상태 기반 처리 
    + 상태의 유형, 갱신 방법, 제거 시점에 따라 세밀한 제어가 필요한 경우 사용 

## 22.4 이벤트 시간 처리의 기본  
+ Creation_Time: 이벤트가 생성된 시간 
+ Arrival_Time: 서버에 도착한 시간 

In [3]:
spark.conf.set("spark.sql.shuffle.partitions", 5) 

val activeDataPath = "../BookSamples/data/activity-data"
val static = spark.read.json(activeDataPath) 
val streaming = spark.readStream.schema(static.schema).option("maxFilesPerTrigger", 10).json(activeDataPath) 

static.printSchema() 

root
 |-- Arrival_Time: long (nullable = true)
 |-- Creation_Time: long (nullable = true)
 |-- Device: string (nullable = true)
 |-- Index: long (nullable = true)
 |-- Model: string (nullable = true)
 |-- User: string (nullable = true)
 |-- gt: string (nullable = true)
 |-- x: double (nullable = true)
 |-- y: double (nullable = true)
 |-- z: double (nullable = true)



activeDataPath = ../BookSamples/data/activity-data
static = [Arrival_Time: bigint, Creation_Time: bigint ... 8 more fields]
streaming = [Arrival_Time: bigint, Creation_Time: bigint ... 8 more fields]


[Arrival_Time: bigint, Creation_Time: bigint ... 8 more fields]

In [4]:
spark.streams.active

Array()

## 22.5 이벤트 시간 윈도우 
+ 나노세컨드 단위의 유닉스 시간으로 표현됨

In [5]:
val withEventTime = streaming.selectExpr("*", "cast(cast(Creation_Time as double)/1000000000 as timestamp) as event_time") 

withEventTime = [Arrival_Time: bigint, Creation_Time: bigint ... 9 more fields]


[Arrival_Time: bigint, Creation_Time: bigint ... 9 more fields]

### 22.5.1 텀블링 윈도우 
+ 디버깅 목적으로 결과를 메모리 싱크에 저장함 
+ 스트림 처리가 시작되면 SQL로 결과 조회가 가능함 
+ windows 컬럼은 struct 타입이며 윈도우 시작과 종료를 의미함 

In [6]:
import org.apache.spark.sql.functions.{window, col} 

val active = withEventTime.groupBy(window(col("event_time"), "10 minutes"))
    .count() 
    .writeStream
    .queryName("events_per_windows") 
    .format("memory") 
    .outputMode("complete") 
    .start()

active = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@4b2c47d2


org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@4b2c47d2

In [7]:
spark.streams.active

Array(org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@4b2c47d2)

In [8]:
spark.sql("select * from events_per_windows").show() 

+--------------------+-----+
|              window|count|
+--------------------+-----+
|[2015-02-23 10:40...|10973|
|[2015-02-24 11:50...|18844|
|[2015-02-24 13:00...|16621|
|[2015-02-23 13:20...|13319|
|[2015-02-22 00:40...|    1|
|[2015-02-23 12:30...|12658|
|[2015-02-24 11:20...|14225|
|[2015-02-23 10:20...|12395|
|[2015-02-24 12:20...|16648|
|[2015-02-24 14:00...|18740|
|[2015-02-24 12:30...|15773|
|[2015-02-24 13:10...|13182|
|[2015-02-24 14:10...|21174|
|[2015-02-23 10:30...|12493|
|[2015-02-24 13:40...|16501|
|[2015-02-24 13:50...|12001|
|[2015-02-23 14:30...|11854|
|[2015-02-23 12:20...|13294|
|[2015-02-23 13:40...|20836|
|[2015-02-23 11:20...| 9408|
+--------------------+-----+
only showing top 20 rows



In [10]:
active.stop()

In [11]:
spark.streams.active

Array()

+ 슬라이딩 윈도우 

In [12]:
import org.apache.spark.sql.functions.{window, col} 

val active = withEventTime.groupBy(window(col("event_time"), "10 minutes", "5 minutes")) 
    .count() 
    .writeStream 
    .queryName("events_per_windows") 
    .format("memory") 
    .outputMode("complete") 
    .start()

active = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@c81bbd0


org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@c81bbd0

In [13]:
spark.sql("select * from events_per_windows").show(20, 0) 

+------------------------------------------+------+
|window                                    |count |
+------------------------------------------+------+
|[2015-02-23 14:15:00, 2015-02-23 14:25:00]|107668|
|[2015-02-24 11:50:00, 2015-02-24 12:00:00]|150773|
|[2015-02-24 13:00:00, 2015-02-24 13:10:00]|133323|
|[2015-02-22 00:35:00, 2015-02-22 00:45:00]|35    |
|[2015-02-23 12:30:00, 2015-02-23 12:40:00]|100853|
|[2015-02-23 10:20:00, 2015-02-23 10:30:00]|99178 |
|[2015-02-23 13:25:00, 2015-02-23 13:35:00]|91684 |
|[2015-02-24 14:25:00, 2015-02-24 14:35:00]|203945|
|[2015-02-23 12:55:00, 2015-02-23 13:05:00]|113953|
|[2015-02-22 00:40:00, 2015-02-22 00:50:00]|35    |
|[2015-02-23 12:35:00, 2015-02-23 12:45:00]|91221 |
|[2015-02-23 13:05:00, 2015-02-23 13:15:00]|205912|
|[2015-02-24 11:20:00, 2015-02-24 11:30:00]|113768|
|[2015-02-24 13:35:00, 2015-02-24 13:45:00]|174863|
|[2015-02-24 14:00:00, 2015-02-24 14:10:00]|150225|
|[2015-02-24 12:30:00, 2015-02-24 12:40:00]|125679|
|[2015-02-24

In [14]:
active.stop()

### 22.5.2 워터마크로 지연 데이터 제어하기 
+ 얼마나 늦게 도착한 데이터까지 받아들일지 기준을 정하지 않았기 때문에 중간 결과 데이터를 영원히 저장하는 문제가 있음 
+ 오래된 데이터를 제거하는 데 필요한 워터마크를 반드시 지정해야 함 
+ 최대 5시간까지 데이터가 지연된다면? 

In [15]:
val active = withEventTime 
    .withWatermark("event_time", "5 hours") 
    .groupBy(window(col("event_time"), "10 minutes", "5 minutes")) 
    .count() 
    .writeStream
    .queryName("events_per_windows") 
    .format("memory") 
    .outputMode("complete") 
    .start()

active = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@318f7c0d


org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@318f7c0d

In [16]:
active.stop()

## 22.6 스트림에서 중복 데이터 제거하기 
+ 상태 정보가 무한히 커지지 않도록 워터마크를 명시해야 함 
+ 예제에서는 중복 이벤트가 같은 식별자와 타임스탬프를 가진다고 가정함 

In [19]:
val active = withEventTime
    .withWatermark("event_time", "5 hours") 
    .dropDuplicates("User", "event_time") 
    .groupBy(window(col("event_time"), "10 minutes", "5 minutes")) 
    .count() 
    .writeStream 
    .queryName("events_per_windows") 
    .format("memory") 
    .outputMode("complete") 
    .start() 

active = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@4fac9781


org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@4fac9781

In [21]:
spark.sql("select * from events_per_windows").show(20, 0) 

+------------------------------------------+-----+
|window                                    |count|
+------------------------------------------+-----+
|[2015-02-23 14:15:00, 2015-02-23 14:25:00]|13484|
|[2015-02-24 11:50:00, 2015-02-24 12:00:00]|37799|
|[2015-02-24 13:00:00, 2015-02-24 13:10:00]|33383|
|[2015-02-22 00:35:00, 2015-02-22 00:45:00]|1    |
|[2015-02-23 12:30:00, 2015-02-23 12:40:00]|12658|
|[2015-02-23 10:20:00, 2015-02-23 10:30:00]|12395|
|[2015-02-23 13:25:00, 2015-02-23 13:35:00]|11441|
|[2015-02-24 14:25:00, 2015-02-24 14:35:00]|50960|
|[2015-02-23 12:55:00, 2015-02-23 13:05:00]|14172|
|[2015-02-22 00:40:00, 2015-02-22 00:50:00]|1    |
|[2015-02-23 12:35:00, 2015-02-23 12:45:00]|11424|
|[2015-02-23 13:05:00, 2015-02-23 13:15:00]|25731|
|[2015-02-24 11:20:00, 2015-02-24 11:30:00]|28394|
|[2015-02-24 13:35:00, 2015-02-24 13:45:00]|43672|
|[2015-02-24 14:00:00, 2015-02-24 14:10:00]|37491|
|[2015-02-24 12:30:00, 2015-02-24 12:40:00]|31366|
|[2015-02-24 13:10:00, 2015-02-

In [22]:
active.stop()

## 22.7 임의적인 상태 기반 처리
+ 다음과 같은 처리 가능
    + 특정 키의 개수를 기반으로 윈도우 생성
    + 특성 시간 범위 안에 일정 개수 이상의 이벤트가 있는 경우 알림 발생시키기
    + 결정되지 않은 시간 동안 사용자 세션을 유지하고 향후 분석을 위해 세션 저장하기

+ 두 가지의 유형
    + mapGroupsWithState API : 최대 한 개의 로우를 만듬
    + flatMapGroupsWithState API : 하나 이상의 로우를 만듬

### 22.7.1 타임아웃
+ 타임 아웃은 전체 그룹에 대한 전역 파라미터로 동작함
+ 타임아웃은 처리 시간(GroupStateTimeout.ProcessingTimeTimeout)이나 이벤트 시간(GroupStateTimeout.EventTimeTimeout) 중 하나가 될 수 있음